# Basic work

### Setup

In [1]:
var moment = require('moment');
var { constants } = require('../utils');
var { advanceTimeAndBlock } = require('../utils/evm');
var { toUnit } = require('../utils/bn');
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm');
var { next, clear } = require('../utils/jupyter');
var { Notebook } = require('../utils/notebook');

clear();

In [2]:
var notebook = new Notebook();

In [3]:
next(async () => {
    await notebook.setup();
});

### Add LP tokens to job

In [4]:
next(async () => {
    await notebook.addLiquidityToJob(toUnit(1));
});

### Work, work, work
Sleep 3 periods, add the same amount of liquidity than last time, sleep 3 periods, work

In [5]:
next(async () => {
    console.log('Start of simulation');
    await notebook.recordCredits();
    
    // sleep 2 periods
    await advanceTimeAndBlock(notebook.rewardPeriod * 3);
    await notebook.recordCredits();
    
    // add liquidity
    await notebook.addLiquidityToJob(toUnit(1));
    await notebook.recordCredits();
    
    // sleep 2 periods
    await advanceTimeAndBlock(notebook.rewardPeriod * 3);
    await notebook.recordCredits();
    
    // work
    await notebook.job.connect(notebook.keeper).work();
    await notebook.recordCredits();
    
    console.log('End of simulation');
});

In [6]:
next(async () => {
    await notebook.draw();
});

Start of simulation
End of simulation
